In [2]:
import csv
import sys

from util import Node, StackFrontier, QueueFrontier

In [3]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}



In [4]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])


    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


In [16]:
load_data("C:\\Users\\Usama Arshad\\Downloads\\degrees\\degrees\\small")

In [17]:
people

{'102': {'name': 'Kevin Bacon',
  'birth': '1958',
  'movies': {'104257', '112384'}},
 '129': {'name': 'Tom Cruise', 'birth': '1962', 'movies': {'104257', '95953'}},
 '144': {'name': 'Cary Elwes', 'birth': '1962', 'movies': {'93779'}},
 '158': {'name': 'Tom Hanks', 'birth': '1956', 'movies': {'109830', '112384'}},
 '1597': {'name': 'Mandy Patinkin', 'birth': '1952', 'movies': {'93779'}},
 '163': {'name': 'Dustin Hoffman', 'birth': '1937', 'movies': {'95953'}},
 '1697': {'name': 'Chris Sarandon', 'birth': '1942', 'movies': {'93779'}},
 '193': {'name': 'Demi Moore', 'birth': '1962', 'movies': {'104257'}},
 '197': {'name': 'Jack Nicholson', 'birth': '1937', 'movies': {'104257'}},
 '200': {'name': 'Bill Paxton', 'birth': '1955', 'movies': {'112384'}},
 '398': {'name': 'Sally Field', 'birth': '1946', 'movies': {'109830'}},
 '420': {'name': 'Valeria Golino', 'birth': '1965', 'movies': {'95953'}},
 '596520': {'name': 'Gerald R. Molen', 'birth': '1935', 'movies': {'95953'}},
 '641': {'name': '

In [9]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """
    #Number of Explored Paths
    num_explored=0
    
    #Start Point and Stack initiation
    start = Node(state=source, parent=None, action=None)
    frontier = StackFrontier()
    frontier.add(start)
    
    #Keep account of explored sets
    explored=()
    
    # Keep looping until solution found
    while True:

        # If nothing left in frontier, then no path
        if frontier.empty():
            raise Exception("no solution")

        # Choose a node from the frontier
        node = frontier.remove()
        num_explored += 1
        
        #Mark Node as Explored
        explored.add(node.state)
        
        #Neighbors addition to the Frontier
        for movie_id, person_id  in neighbors_for_person(node.state):
            if not frontier.contains_state(person_id) and person_id not in explored:
                child = Node(state=person_id, parent=node, action=movie_id)
                frontier.add(child)
                
                #Checking if node is the goal then we have a solution
                if child.state == target:
                    movies = []
                    people = []
                    solution=[]
                    
                    while child.parent is not None:
                        movies.append(child.action)
                        people.append(child.state)
                        child = node.parent
                    movies.reverse()
                    people.reverse()
                    a = zip(movies, people)
                    for movie,person in x:
                        solution.append(movie,person)
                    return solution


In [7]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]




In [8]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors




In [ ]:

if __name__ == "__main__":
    main()

In [80]:
source=neighbors_for_person('158')
source

{('109830', '158'),
 ('109830', '398'),
 ('109830', '641'),
 ('109830', '705'),
 ('112384', '102'),
 ('112384', '158'),
 ('112384', '200'),
 ('112384', '641')}

In [61]:
#Checking the case when they directly worked together in the same movie

In [136]:
a='158'
b='201'

In [137]:
source=neighbors_for_person('158')
target=neighbors_for_person('200')

In [138]:
source=list(source)
target=list(target)

In [139]:
print(f"Source: {source} \n Taget: {target}")

Source: [('109830', '641'), ('109830', '158'), ('109830', '705'), ('112384', '102'), ('109830', '398'), ('112384', '200'), ('112384', '158'), ('112384', '641')] 
 Taget: [('112384', '200'), ('112384', '158'), ('112384', '641'), ('112384', '102')]


In [140]:
j = []
o=[]

condition_met = False

for i in range(len(source)):
    if source[i][1] == b:
        j.append(source[i])
        condition_met = True  # Set flag to True if condition is satisfied

# Second loop: Only run if the 'if' condition was never satisfied
if not condition_met:
    for m in range(len(source)):
        neighbors = list(neighbors_for_person(source[m][1]))
        neighbors.sort()  # Sort neighbors if you want consistent order
        j.append(neighbors)
    for x in range(len(j)):
        for y in range len(j[x]):
            if j[x][y]==b:
                o.append(j[x][y])
                
            


In [143]:
def comparison(source_id,target_id):
    source=neighbors_for_person(source_id)
    source=list(source)
    j=[]
    for i in range(len(source)):
        if source[i][1] == target_id:
            j.append(source[i])
            return j
        else:
            return None
